In [1]:
import math
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [6]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel("RUL_SN_1_optmizable", modelRULSN, selected_features, dataFolder = '../CMAPSSData',
                          scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.trainTime
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateModel(["rhs"], round=2)
    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

In [7]:
file = open("testFile.txt", "w") 

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

#res = nn_optmize_fun(np.array([30, 1, 125]), selected_features = selected_features, scaler = min_max_scaler, 
#                     datasetNumber = '1', verbose=1, epochs=20, saveToFile = file)

file.close()
#print(res)

In [8]:
#Optimize the parameters for the NN using DE

maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
totalTime = {'1':0, '2':0, '3':0, '4':0}
results = {'1':0, '2':0, '3':0, '4':0}

#maxWindowSize = {'4':18}
datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

for datasetNumber in maxWindowSize:
    
    print("Tunning for dataset "+datasetNumber)
    file = open("intermediateResults2"+datasetNumber+".txt", "w")

    windowSizeBounds = [1,maxWindowSize[datasetNumber]]
    windowStrideBounds = [1,10]
    constantRULBounds = [90,140]

    bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]


    startTime = time.clock()
    tempResults = differential_evolution(nn_optmize_fun, bounds, 
                                     args=(selected_features, datasetNumber, min_max_scaler, 0, 20, file, 1),
                                    strategy='best1bin', maxiter=10, popsize=10, disp=True)
    results[datasetNumber] = tempResults
    endTime = time.clock()

    file.close()
    totalTime[datasetNumber] = endTime - startTime

Tunning for dataset 2
259/259 [==============================] - 0s 116us/step
differential_evolution step 1: f(x)= 30.8518
259/259 [==============================] - 0s 116us/step
differential_evolution step 2: f(x)= 30.8518
259/259 [==============================] - 0s 119us/step
differential_evolution step 3: f(x)= 30.3716
259/259 [==============================] - 0s 139us/step


259/259 [==============================] - 0s 123us/step
differential_evolution step 4: f(x)= 30.3716
259/259 [==============================] - 0s 119us/step
differential_evolution step 5: f(x)= 30.3354
259/259 [==============================] - 0s 123us/step
Tunning for dataset 1
100/100 [==============================] - 0s 249us/step
differential_evolution step 1: f(x)= 14.8101
100/100 [==============================] - 0s 239us/step
differential_evolution step 2: f(x)= 14.8101
100/100 [==============================] - 0s 250us/step
differential_evolution step 3: f(x)= 14.738
100/100 [==============================] - 0s 249us/step


100/100 [==============================] - 0s 249us/step
differential_evolution step 4: f(x)= 14.738
100/100 [==============================] - 0s 259us/step
differential_evolution step 5: f(x)= 14.5818
100/100 [==============================] - 0s 249us/step
differential_evolution step 6: f(x)= 14.5818
100/100 [==============================] - 0s 249us/step
differential_evolution step 7: f(x)= 14.5818
100/100 [==============================] - 0s 249us/step
differential_evolution step 8: f(x)= 14.5818
100/100 [==============================] - 0s 249us/step
differential_evolution step 9: f(x)= 14.5818
100/100 [==============================] - 0s 289us/step
Tunning for dataset 3
100/100 [==============================] - 0s 259us/step


100/100 [==============================] - 0s 249us/step
differential_evolution step 1: f(x)= 15.7245
100/100 [==============================] - 0s 249us/step
differential_evolution step 2: f(x)= 15.7245
100/100 [==============================] - 0s 249us/step
differential_evolution step 3: f(x)= 15.6976
100/100 [==============================] - 0s 249us/step
differential_evolution step 4: f(x)= 15.0515
100/100 [==============================] - 0s 249us/step
differential_evolution step 5: f(x)= 15.0515
100/100 [==============================] - 0s 249us/step
differential_evolution step 6: f(x)= 14.7587
100/100 [==============================] - 0s 249us/step
differential_evolution step 7: f(x)= 14.7587
100/100 [==============================] - 0s 259us/step
differential_evolution step 8: f(x)= 14.7587
100/100 [==============================] - 0s 259us/step
differential_evolution step 9: f(x)= 14.7587
100/100 [==============================] - 0s 272us/step


100/100 [==============================] - 0s 249us/step
differential_evolution step 10: f(x)= 14.7587
100/100 [==============================] - 0s 249us/step
Tunning for dataset 4
248/248 [==============================] - 0s 117us/step
differential_evolution step 1: f(x)= 36.8221
248/248 [==============================] - 0s 113us/step
differential_evolution step 2: f(x)= 36.8186
248/248 [==============================] - 0s 109us/step
differential_evolution step 3: f(x)= 36.8186
248/248 [==============================] - 0s 129us/step
differential_evolution step 4: f(x)= 36.8186
248/248 [==============================] - 0s 113us/step


248/248 [==============================] - 0s 113us/step
differential_evolution step 5: f(x)= 36.6938
248/248 [==============================] - 0s 117us/step
differential_evolution step 6: f(x)= 36.6938
248/248 [==============================] - 0s 117us/step
differential_evolution step 7: f(x)= 36.6938
248/248 [==============================] - 0s 112us/step
differential_evolution step 8: f(x)= 36.6938
248/248 [==============================] - 0s 112us/step


In [9]:
print("Total time {}".format(totalTime))
print(results)

Total time {'2': 2745.194047008363, '1': 1388.363804037871, '3': 1816.3322229449814, '4': 3488.433744119323}
{'2':      fun: 30.33536691708928
 message: 'Optimization terminated successfully.'
    nfev: 304
     nit: 5
 success: True
       x: array([  5.95736308,   1.85726437, 125.9387272 ]), '1':      fun: 14.581765584909196
 message: 'Optimization terminated successfully.'
    nfev: 380
     nit: 9
 success: True
       x: array([ 25.44397545,   1.65054251, 128.63195976]), '3':      fun: 14.75865424611616
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 422
     nit: 10
 success: False
       x: array([ 29.9569236 ,   1.52439302, 137.91017404]), '4':      fun: 36.69382626152096
 message: 'Optimization terminated successfully.'
    nfev: 354
     nit: 8
 success: True
       x: array([ 17.2569061 ,   1.69885032, 131.35899462])}


In [1]:
#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

res = nn_optmize_fun(np.array([25, 2, 105]), selected_features = selected_features, scaler = min_max_scaler, 
                     datasetNumber = '1', verbose=1, epochs=20, saveToFile = None)

print(res)

NameError: name 'MinMaxScaler' is not defined

In [17]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plot(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()